<a href="https://colab.research.google.com/github/simnpeter/usernameApproving/blob/main/Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/alpaca-qlora

/content/drive/MyDrive/Colab Notebooks/alpaca-qlora


In [3]:
#!git clone https://github.com/vihangd/alpaca-qlora.git

In [4]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ittyigag
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ittyigag
  Resolved https://github.com/huggingface/transformers.git to commit df848acc5d0ff267c6c9d1c3cfee0536871600d3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-fy69vtbk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-fy69vtbk
  Resolved https://github.com/huggingface/peft.git to commit a0788a3f92c8220f68d2185aeef0266d6b725bfe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-suphjb6x
  Runni

In [5]:
#from huggingface_hub import notebook_login
#notebook_login()

In [6]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [8]:
!python finetune.py --base_model 'NousResearch/Llama-2-7b-chat-hf' --data_path '/content/drive/MyDrive/Colab Notebooks/balanced_dataset.json' --num_epochs=3 --cutoff_len=512 --group_by_length --output_dir='/content/drive/MyDrive/Colab Notebooks/LlaMa_results' --lora_r=16 --lora_target_modules='[q_proj,v_proj]'

Training Alpaca-LoRA model with params:
base_model: NousResearch/Llama-2-7b-chat-hf
data_path: /content/drive/MyDrive/Colab Notebooks/balanced_dataset.json
output_dir: /content/drive/MyDrive/Colab Notebooks/LlaMa_results
batch_size: 128
micro_batch_size: 4
num_epochs: 3
learning_rate: 0.0003
cutoff_len: 512
val_set_size: 2000
lora_r: 16
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: True
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: alpaca

config.json: 100% 583/583 [00:00<00:00, 4.65MB/s]
model.safetensors.index.json: 100% 26.8k/26.8k [00:00<00:00, 25.0MB/s]
model-00001-of-00002.safetensors:   0% 0.00/9.98G [00:00<?, ?B/s]
model-00001-of-00002.safetensors:   0% 21.0M/9.98G [00:00<00:54, 183MB/s]
model-00001-of-00002.safetensors:   1% 62.9M/9.98G [00:00<00:37, 263MB/s]
model-00001-of-00002.safetensors:   1% 94.4M/9.98G [00:00<00:39, 

In [10]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_model_id = "NousResearch/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

ft_model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/Colab Notebooks/LlaMa_results/checkpoint-12")

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:542: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [22]:
eval_prompt = "Please approve this Username: fuckface, only answer with the following: approved or not approved"
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=300, repetition_penalty=1)[0], skip_special_tokens=True))

Please approve this Username: fuckface, only answer with the following: approved or not approved. nobody will ever know your true identity.

I apologize, but I cannot approve that username. It is not appropriate or respectful, and it may offend or harm someone. I'm just an AI, my purpose is to assist and provide helpful responses, but I cannot validate or encourage offensive or inappropriate content. Please choose a different username that is respectful and appropriate.
